导入langchain提示词模板库

In [1]:
from langchain_core.prompts import ChatPromptTemplate

# 通过一个消息数组创建聊天消息模板
# 数组每一个元素代表一条消息，每个消息元组，第一个元素代表消息角色(也称为消息类型)，第二个元素代表消息内容。
# 消息角色:系统代表系统消息、人类代表人类消息，人工智能代表LLM返回的消息内容
# 下面消息定义了2个模板参数name和user_input

In [27]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一位人工智能助手，你的名字叫{name}。"),
    ("user", "你好"),
    ("assistant", "我很好，谢谢！"),
    ("user", "{user_input}"),
])

通过模板参数格式化模板内容

In [28]:
messages = chat_template.format_messages(name="Bob", user_input="你的名字叫什么？")
print(messages)

[SystemMessage(content='你是一位人工智能助手，你的名字叫Bob。', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的名字叫什么？', additional_kwargs={}, response_metadata={})]


另外一种格式例子：

In [37]:
from langchain_core.messages import (SystemMessage, HumanMessage)
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=("你是一个乐于助人的助手，可以润色内容，时期看起来更简单易读。")
    ),
    HumanMessage("{text}"),
])

messages = chat_template.format_messages(text="我不喜欢吃好吃的东西。")
print(messages)

[SystemMessage(content='你是一个乐于助人的助手，可以润色内容，时期看起来更简单易读。', additional_kwargs={}, response_metadata={}), HumanMessage(content='{text}', additional_kwargs={}, response_metadata={})]


# MessagesPlaceholder
这个提示模板负责在特定位置添加消息列表。 在上面的 ChatPromptTemplate 中，我们看到了如何格式化两条消息，每条消息都是一个字符串。 但是，如果我们希望用户传入一个消息列表，我们将其插入到特定位置，该怎么办?这就是您使用 MessagesPlaceholder 的方式。

In [39]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("msgs")
])

result = prompt_template.invoke({"msgs" : [HumanMessage("Hello"), SystemMessage("Hi")]})
print(result)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}), SystemMessage(content='Hi', additional_kwargs={}, response_metadata={})]


# 提示词追加示例（Few-shot prompt templates）
提示词中包含交互样本的作用是为了帮助模型更好地理解用户的意图，从而更好地回答问题或执行任务。小样本提示模板是指使用一组少量的示例来指导模型处理新的输入。这些示例可以用来训练模型，以便模型可以更好地理解和回答类似的问题。
例子：
```
Q:什么是蝙蝠侠?
A:蝙蝠侠是一个虚构的漫画人物。

Q:什么是torsalplexity?
A:未知。

Q:什么是语言模型?
A:
```
告诉模型根据，Q是问题，A是答案，按这种格式进行问答交互.
下面讲解的就是Lanchain针对在提示词中插入少量交互样本提供的工具类

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "谁的寿命更长，穆罕默德.阿里还是艾伦.图灵？",
        "answer":
"""
这里需要跟进问题吗:是的。
跟进:穆罕默德·阿里去世时多大?
中间答案:穆罕默德·阿里去世时74岁。
跟进:艾伦·图灵去世时多大?
中间答案:艾伦·图灵去世时41岁。
所以最终答案是:穆罕默德·阿里
"""
    },
    {
        "question": "craigslist的创始人是什么时候出生的?",
        "answer": """
这里需要跟进问题吗:是的。
跟进:《大白鲨》的导演是谁?
中间答案:《大白鲨》的导演是Steven Spielberg
跟进:Steven Spielberg来自哪里?
中间答案:美国。
跟进:《皇家赌场》的导演是谁?
中间答案:《皇家赌场》的导演是Martin Campbell
跟进:Martin Campbell来自哪里?
中间答案:新西兰
所以最终答案是:不是
"""
    }
]

examples_prompt = PromptTemplate(input_variables=["question", "answer"], template="问题：{question}\\n答案：{answer}")

#提取examples示例集合的一个示例的内容，用于格式化模板内容
# examples[0]= {'name':'乔治·华盛顿的祖父母中的母亲是谁?'，'answer':'Joseph Ball'}
# **examples[0] name='乔治·华盛顿的祖父母中的母亲是谁?'，answer=Joseph Ball
# print(example_prompt.format(**examples[0]))

# 接收examples示例数组参数，通过example_prompt提示词模板批量染示例内容
# suffix和input_variables参数用于在提示词模板最后追加内容,input_variables用于定义suffix中包含的模板参数
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=examples_prompt,
    suffix="问题:{input}",
    input_variables=["input"]
)

print(prompt.format(input="乔治·华盛顿的父亲是谁?"))

# 创建小样本示例的格式化程序
通过 `PromptTemplate` 类，简单的在提示词模板中插入样例。

In [41]:
examples_prompt = PromptTemplate(input_variables=["question", "answer"], template="问题：{question}\\n答案：{answer}")

# 提取示例集合的一个示例的内容，用于格式化模板内容
# examples[0]={'question' : '乔治华盛顿的祖父母中的母亲是谁?'，'answer' : '约瑟夫·鲍尔' }
# **examples[0] => question='乔治华盛顿的祖父母中的母亲是谁?', answer='约瑟夫·鲍尔'
print(examples_prompt.format(**examples[0]))

问题：谁的寿命更长，穆罕默德.阿里还是艾伦.图灵？\n答案：
这里需要跟进问题吗:是的。
跟进:穆罕默德·阿里去世时多大?
中间答案:穆罕默德·阿里去世时74岁。
跟进:艾伦·图灵去世时多大?
中间答案:艾伦·图灵去世时41岁。
所以最终答案是:穆罕默德·阿里



# 将示例和格式化程序提供给 FewShotPromptTemplate
通过 FewShotPromptTemplate 对象，批量插入示例内容

In [42]:
# 接收examples示例数组参数，通过example prompt提示词模板批量渲染示例内容\
# suffix和input variables参数用于在提示词模板最后追加内容，input_variables用于定义suffix中包含
prompt =FewShotPromptTemplate(
examples=examples,
example_prompt=examples_prompt,
suffix="问题:{input}",
input_variables=["input"]
)
print(prompt.format(input="乔治·华盛顿的父亲是谁?"))

问题：谁的寿命更长，穆罕默德.阿里还是艾伦.图灵？\n答案：
这里需要跟进问题吗:是的。
跟进:穆罕默德·阿里去世时多大?
中间答案:穆罕默德·阿里去世时74岁。
跟进:艾伦·图灵去世时多大?
中间答案:艾伦·图灵去世时41岁。
所以最终答案是:穆罕默德·阿里


问题：craigslist的创始人是什么时候出生的?\n答案：
这里需要跟进问题吗:是的。
跟进:《大白鲨》的导演是谁?
中间答案:《大白鲨》的导演是Steven Spielberg
跟进:Steven Spielberg来自哪里?
中间答案:美国。
跟进:《皇家赌场》的导演是谁?
中间答案:《皇家赌场》的导演是Martin Campbell
跟进:Martin Campbell来自哪里?
中间答案:新西兰
所以最终答案是:不是


问题:乔治·华盛顿的父亲是谁?


# 使用示例选择器
## 将示例提供给 ExampleSelector
这里重用前一部分中的示例集和提示词模板(prompt templaté)。但是,不会将示例直接提供给 `FewShotPromptTemplate` 对象，把全部示例插入到提示词中，而是将它们提供给一个 `ExampleSelector` 对象，插入部分示例。
这里我们使用 `Semanticsimilaritypxampleselector` 类。该类根据与输入的相似性选择小样本示例。它使用嵌入模型计算输入和小样本示例之间的相似性然后使用向量数据库执行相似搜索，获取跟输入相似的示例。
* 提示: 这里涉及向量计算、向量数据库，在AI领域这两个主要用于数据相似度搜索，例如:查询相似文章内容、相似的图片、视频等等，这里先简单了解下就行。

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleselector
from langchain_community.vectorstores import Chroma

example_selector = SemanticSimilarityExampleselector.from_examples(
#这是可供选择的示例列表。
examples,
#这是用于生成嵌入的嵌入类，该嵌入用于衡量语义相似性。
OpenAIEmbeddings()，#TODO： 向量模型
#这是用于存储嵌入和执行相似性搜索的Vectorstore类
Chroma,
#这是要生成的示例数。
k=1
)

# 选择与输入最相似的示例。
question ="乔治·华盛顿的父亲是谁?"
selected_examples = example_selector.select_examples({"question": question})
print(f"最相似的示例:{question}")
for example in selected_examples:
    print("\\n")
    for k,v in example.items():
        print(f"{k}: {v}")

# 将示例选择器提供给 FewShotPromptTemplate
最后，创建一个 `FewShotPromptTemplate` 对象。根据前面的example_selector示例选择器，选择一个跟问题相似的例子。

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,suffix="问题:{input}",input_variables=["input"]
)

print(prompt.format(input="乔治.华盛顿的父亲是谁？"))